# Multi-Agent System with Supervisor

This notebook demonstrates a multi-agent architecture where a **Supervisor Agent** coordinates multiple specialized worker agents.

## Architecture:
- **Supervisor Agent**: Decides which agent should act next based on the current state
- **Research Agent**: Searches for information on the web
- **Code Agent**: Writes and explains code
- **Writer Agent**: Creates written content

## Prerequisites
Install required packages:
```bash
pip install langgraph langchain langchain-openai langchain-anthropic
```

In [ ]:
# Import required libraries
from typing import Annotated, Literal, TypedDict
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END, START
from langgraph.prebuilt import create_react_agent
import functools
import operator

In [ ]:
# Set your API key
import os
os.environ["ANTHROPIC_API_KEY"] = "your-api-key-here"

## Step 1: Define the State

The state contains all information shared between agents.

In [ ]:
class AgentState(TypedDict):
    """The state of our multi-agent system"""
    messages: Annotated[list, operator.add]  # Messages accumulate over time
    next_agent: str  # Which agent should act next
    task: str  # The original task

## Step 2: Create Worker Agents

Each agent has a specific role and capabilities.

In [ ]:
# Initialize the LLM
llm = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0)

# Define agent nodes
def create_agent_node(agent_name: str, system_message: str):
    """Creates an agent node with a specific system message"""
    
    def agent_node(state: AgentState):
        messages = [
            SystemMessage(content=system_message),
            HumanMessage(content=state["task"])
        ] + state["messages"]
        
        response = llm.invoke(messages)
        
        return {
            "messages": [HumanMessage(content=f"[{agent_name}]: {response.content}")]
        }
    
    return agent_node

# Create specialized agents
research_agent = create_agent_node(
    "Research Agent",
    "You are a research agent. Your job is to gather information and provide factual, "
    "well-researched answers. Focus on accuracy and cite sources when possible."
)

code_agent = create_agent_node(
    "Code Agent",
    "You are a coding expert. Your job is to write clean, efficient code with proper "
    "documentation. Explain your code clearly and follow best practices."
)

writer_agent = create_agent_node(
    "Writer Agent",
    "You are a professional writer. Your job is to create engaging, well-structured "
    "content. Focus on clarity, tone, and proper formatting."
)

## Step 3: Create the Supervisor Agent

The supervisor analyzes the task and decides which agent should handle it.

In [ ]:
def supervisor_node(state: AgentState):
    """The supervisor decides which agent should act next or if we're done"""
    
    supervisor_prompt = f"""You are a supervisor managing a team of agents:
    
- Research Agent: Best for gathering information, facts, and research
- Code Agent: Best for writing code, technical implementations, and programming tasks
- Writer Agent: Best for creating articles, documentation, and written content
- FINISH: Use this when the task is complete

Current task: {state['task']}

Conversation history:
{state['messages']}

Based on the task and conversation history, which agent should act next?
Respond with ONLY ONE of: Research Agent, Code Agent, Writer Agent, or FINISH
"""
    
    response = llm.invoke([HumanMessage(content=supervisor_prompt)])
    
    # Parse the response to determine next agent
    next_agent = response.content.strip()
    
    # Map agent names to node names
    agent_map = {
        "Research Agent": "research",
        "Code Agent": "code",
        "Writer Agent": "writer",
        "FINISH": "FINISH"
    }
    
    # Find the matching agent
    next_node = "FINISH"
    for agent_name, node_name in agent_map.items():
        if agent_name.lower() in next_agent.lower():
            next_node = node_name
            break
    
    print(f"\n🎯 Supervisor Decision: {next_agent} -> {next_node}")
    
    return {"next_agent": next_node}

## Step 4: Build the Graph

Connect all agents in a graph where the supervisor routes to appropriate agents.

In [ ]:
# Create the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("supervisor", supervisor_node)
workflow.add_node("research", research_agent)
workflow.add_node("code", code_agent)
workflow.add_node("writer", writer_agent)

# Define the routing function
def route_to_agent(state: AgentState) -> Literal["research", "code", "writer", "__end__"]:
    """Route to the next agent based on supervisor's decision"""
    if state["next_agent"] == "FINISH":
        return "__end__"
    return state["next_agent"]

# Set up the edges
workflow.add_edge(START, "supervisor")
workflow.add_conditional_edges(
    "supervisor",
    route_to_agent,
    {
        "research": "research",
        "code": "code",
        "writer": "writer",
        "__end__": END
    }
)

# After each agent finishes, go back to supervisor
workflow.add_edge("research", "supervisor")
workflow.add_edge("code", "supervisor")
workflow.add_edge("writer", "supervisor")

# Compile the graph
app = workflow.compile()

print("✅ Multi-agent system compiled successfully!")

## Step 5: Visualize the Graph (Optional)

Visualize the agent workflow.

In [ ]:
# Visualize the graph structure
try:
    from IPython.display import Image, display
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not display graph: {e}")
    print("Install graphviz if you want to visualize the graph")

## Step 6: Run the Multi-Agent System

Let's test the system with different tasks.

In [ ]:
def run_task(task: str):
    """Run a task through the multi-agent system"""
    print(f"\n{'='*80}")
    print(f"📋 TASK: {task}")
    print(f"{'='*80}\n")
    
    initial_state = {
        "messages": [],
        "task": task,
        "next_agent": ""
    }
    
    # Run the workflow
    result = app.invoke(initial_state)
    
    print("\n" + "="*80)
    print("📊 RESULTS:")
    print("="*80)
    for msg in result["messages"]:
        print(f"\n{msg.content}\n")
    
    return result

### Example 1: Coding Task

In [ ]:
result1 = run_task(
    "Write a Python function that calculates the fibonacci sequence up to n numbers"
)

### Example 2: Research Task

In [ ]:
result2 = run_task(
    "Explain what LangGraph is and how it differs from LangChain"
)

### Example 3: Writing Task

In [ ]:
result3 = run_task(
    "Write a brief blog post introduction about the benefits of multi-agent systems in AI"
)

### Example 4: Complex Task (Multiple Agents)

In [ ]:
result4 = run_task(
    "Research what a binary search tree is, then write code to implement it, "
    "and finally write a brief explanation of how it works"
)

## Key Concepts

### 1. **Supervisor Pattern**
   - The supervisor agent acts as a router/orchestrator
   - It analyzes the task and conversation history
   - Decides which specialized agent should handle the next step

### 2. **State Management**
   - Shared state between all agents
   - Messages accumulate to maintain context
   - The supervisor uses this context for routing decisions

### 3. **Conditional Edges**
   - Dynamic routing based on supervisor decisions
   - Agents can be called multiple times in sequence
   - Process ends when supervisor returns "FINISH"

### 4. **Scalability**
   - Easy to add new specialized agents
   - Just create a new agent node and update supervisor's routing logic
   - Each agent can have its own tools and capabilities

## Extending the System

You can enhance this system by:
- Adding more specialized agents (data analyst, translator, etc.)
- Giving agents access to tools (web search, calculators, databases)
- Implementing memory systems for long-term context
- Adding human-in-the-loop approval steps
- Creating sub-supervisors for hierarchical agent systems